## find lane pixels

In [2]:
def find_lane_pixels(image): #image --> after warping image
    histogram = np.sum(image[image.shape[0] // 2:, :], axis=0)
    out_img = np.dstack((image, image, image)) * 255 #input image is 1 channel so convert it to 3 channel
    # out_img will be the warped image with the sliding window on it 
    midpoint = np.int(histogram.shape[0] // 2) # get midpoint of histogram
    leftx_base = np.argmax(histogram[:midpoint]) # get index of peak in the first half of histogram which will correspond to the position of left lane
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint # get index of peak in the second half of histogram which will correspond to the position of right lane

    # plt.title('Histogram', fontsize=16)
    # plt.xlabel('Pixel position')
    # plt.ylabel('Counts')
    # plt.plot(histogram);

    nwindows = 9 # no of windows 
    margin = 100 # width of window /2
    minpix = 50 #?????????????????????????????????????????????????????????????????????????????????????

    window_height = np.int(image.shape[0] // nwindows) # get window_height by dividing the image height / nwindows
    # print("image:",image)
    # nonzero(): Return : [tuple_of_arrays] Indices of elements that are non-zero
    nonzero = image.nonzero()        # to get the location of nonzero pixels in image
    # print("image.nonzero: ",nonzero)
    # [[0,5,0,0,1],
    # [0,0,0,0,0],
    # [4,7,-3,0,0]]
    nonzeroy = np.array(nonzero[0])  # to get the rows that have non zero           [0,0,2,2,2]
    nonzerox = np.array(nonzero[1])  # to get the index of each non zero element    [1,4,0,1,2]
    # print("nonzeroy: ",nonzero[0])
    # print("nonzerox: ",nonzero[1])

    # let the start point for window sliding is @ the 2 peaks of histogram
    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        #first window
        # Identify window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window + 1) * window_height  # y of top edge of window
        win_y_high = image.shape[0] - window * window_height       # y of bottom edge of window
        win_xleft_low = leftx_current - margin                     # x of left edge of left window
        win_xleft_high = leftx_current + margin                    # x of right edge of left window
        win_xright_low = rightx_current - margin                   # x of left edge of right window
        win_xright_high = rightx_current + margin                  # x of right edge of right window

        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 4)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 4)

        # Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        #good_left_inds -> array contain indices of non zero values inside the left window region
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        #good_right_inds -> array contain indices of non zero values inside the right window region
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:#>
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:#>
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


## remove_shadow_noise

In [ ]:
def remove_noise(img):
    h=img.shape[0]
    w=img.shape[1]
    half_w=int(w/2)
    polygons = np.array([[(half_w-300,h),(half_w+375,h),(half_w+380,0),(half_w-280,0)]]) #250
    mask=np.ones_like(img)
    cv2.fillPoly(mask,polygons,0)
    masked_image = cv2.bitwise_and(mask,img)
    return masked_image


## search around poly

In [1]:
def search_around_poly(image):
    margin = 100

    # Grab activated pixels
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(image)
###############################################
    window_img = np.copy(out_img)
    filled_lanes_img = np.copy(out_img) 
    if ((len(leftx) == 0) or (len(rightx) == 0) or (len(righty) == 0) or (len(lefty) == 0)):   ####### msh fahmin #########
        out_img = np.dstack((image, image, image)) * 255
        left_curverad = 0
        right_curverad = 0
        center_diff =0
    else:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        ### Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###
        left_lane_inds = ((nonzerox > (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy +
                                       left_fit[2] - margin)) & (nonzerox < (left_fit[0] * (nonzeroy ** 2) +
                                                                             left_fit[1] * nonzeroy + left_fit[
                                                                                 2] + margin)))
        right_lane_inds = ((nonzerox > (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy +
                                        right_fit[2] - margin)) & (nonzerox < (right_fit[0] * (nonzeroy ** 2) +
                                                                               right_fit[1] * nonzeroy + right_fit[
                                                                                   2] + margin)))

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit new polynomials
        left_fitx, right_fitx, ploty = fit_poly(image.shape, leftx, lefty, rightx, righty)


        ym_per_pix = 30 / 720  # meters per pixel in y dimension
        xm_per_pix = 3 / 500  # meters per pixel in x dimension
        # xm_per_pix = 3.7 / 650  # meters per pixel in x dimension
        # xm_per_pix = 3.7/730 # meters per pixel in x dimension
        # xm_per_pix = 3.7/960 # meters per pixel in x dimension
        # Calculate the curvature radii
        left_fit_cr = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2) #left_fit_circle
        right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)
        y_eval = np.max(ploty)
        left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
            2 * left_fit_cr[0])
        right_curverad = ((1 + (
                    2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
            2 * right_fit_cr[0])
                    
#################### added to calculate center offset ##########################################
        camera_center = (left_fitx[-1] + right_fitx[-1])/2
        center_diff = (camera_center-image.shape[1]/2)*xm_per_pix

        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((image, image, image)) * 255
        # window_img = np.zeros_like(out_img)
#######################################################################################
# we need to draw two lanes by fitting 2 line through the points
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        #image = cv2.polylines(image, [pts], False, (0,255,0), 10)
        # x = np.array([0.0, 1.0, 2.0, 3.0,  4.0,  5.0]) 
        # y = np.array([0.0, 0.8, 0.9, 0.1, -0.8, -1.0]) 
        # z = np.polyfit(x, y, 2)
####################################################################################### 
        filled_lanes_img = np.copy(out_img) 

        # Generate and draw a poly to illustrate the lane area
        left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        points = np.hstack((left, right))
        out_img = cv2.fillPoly(out_img, np.int_(points), (0, 255, 255))
        
        #out img is the warped image with the lanes on it left(red) right(blue)
    return out_img, window_img, filled_lanes_img, left_curverad, right_curverad,center_diff
